In [206]:
def f(x, y =1):
    """ General method that returns the value of the function x² + xy + Cy² or x² + C y², depending on the value of h """
    if h % 4 == 3:
        C = int((h+1)/4)
        return x ** 2 + x * y + C * y ** 2
    else:
        # this covers the case when h equals 1 or 2
        C = h
        return x ** 2 + C * y ** 2

def is_prime(n):
    """ Checks if a number is prime by (regular) trial division """
    if n == 2 or n == 3: return True
    if n < 2 or n % 2 == 0: return False
    if n < 9: return True
    if n % 3 == 0: return False
    r = int(n ** 0.5)
    t = 5
    while t <= r:
        if n % t == 0: return False
        if n % (t + 2) == 0: return False
        t += 6
    return True



def find_x_values_composites(x_max):
    """ 
        Finds all the x-values of the composites of f(x) defined above, using the following properties. 
        When f(x) = p*q, 
            f(x+k*q) is divisble by q (and another divisor, f(x+q)/q) for each k
        Goes first through all x and then will continue to create more composites until everything is treated. 
        The cutoff value makes sure that the sieve stops after certain cutoff, namely x_max.  
    """    
    err_x_pos = [] # list to track the x-value of all the composites
    err_q = []
    counter = 0
    
    
    for x in range(-round(1.2*x_max**0.5), round(1.2*x_max**0.5)): # only need to evaluate x up to a bit more than sqrt(x) as the f(x) is quadratic
        if (x+f(x)) < x_max: # as long as the first composite, on f(x)+x is not too big, search for composites
            if f(x) != 1 and f(x) != f(x+f(x)): # For some small h, we need to filter out some edge cases 
                err_x_pos.append(x+f(x)) # add the first x-position to treat to the error list
                err_q.append(f(x)) # add the divisor q, equaling f(x) to the error list

            # now start to find more composites, until done
            while counter < len(err_x_pos): # while there are still composites to treat
                k = 1 # start a counter to generate all the composites on x + k*q
                while err_x_pos[counter] + k * err_q[counter] < x_max: # add those composites only that don't exceed the cutoff value
                    pos = err_x_pos[counter] + k * err_q[counter] 
                    if pos < x_max: 
                        q = f(pos)//err_q[counter] 
                        err_x_pos.append(pos) 
                        err_q.append(q)
                    k += 1
                counter+=1
    return err_x_pos


   
def get_primes(composites_x, x_max):
    """ Calculates the primes """
    primes = []
    for i in range(1, x_max):
        if not (i in composites_x):
            primes.append(f(i))
    return primes

def check_primes(composites_x, x_max):
    """ Checks if there are any composites missed """
    errors = []
    for i in range(1, x_max):
        if not (i in composites_x):
            if not(is_prime(f(i))):
                errors.append(f(i))
    return errors

def check_composites(composites_x):
    """ Checks if there are any composites (working with x-value) that are actually primes """
    mistakes = []
    for composite in composites_x:
        if is_prime(f(composite)):
            mistakes.append(f(composite))
    return mistakes



# The 9 Heegner Numbers are 1, 2, 3, 7, 11, 19, 43, 67 and 163. For all those, extended sieving works
# based on the chosen h, the algorithm will adapt itself to create the right function

cutoff_value = 2000
for h in range(0,15):
    print(f"Evaluating h={h}")
    composites_x_values = find_x_values_composites(cutoff_value)
    
    if 0:
        print('Detected composites (x-position):')
        composites_x_values.sort()
        print(composites_x_values)
    
    mistakes = check_primes(composites_x_values, cutoff_value)
    if(mistakes):    
        print('Calculated primes that are not actually prime:')
        mistakes.sort()
        print(mistakes)
    
    mistakes = check_composites(composites_x_values)
    if(mistakes):
        print('Calculated composites that are actually prime:')
        print(mistakes)

Evaluating h=0
Calculated primes that are not actually prime:
[1, 4, 9, 16, 25, 49, 64, 81, 121, 169, 256, 289, 361, 529, 625, 729, 841, 961, 1024, 1369, 1681, 1849, 2209, 2401, 2809, 3481, 3721, 4096, 4489, 5041, 5329, 6241, 6561, 6889, 7921, 9409, 10201, 10609, 11449, 11881, 12769, 14641, 15625, 16129, 16384, 17161, 18769, 19321, 22201, 22801, 24649, 26569, 27889, 28561, 29929, 32041, 32761, 36481, 37249, 38809, 39601, 44521, 49729, 51529, 52441, 54289, 57121, 58081, 59049, 63001, 65536, 66049, 69169, 72361, 73441, 76729, 78961, 80089, 83521, 85849, 94249, 96721, 97969, 100489, 109561, 113569, 117649, 120409, 121801, 124609, 128881, 130321, 134689, 139129, 143641, 146689, 151321, 157609, 160801, 167281, 175561, 177241, 185761, 187489, 192721, 196249, 201601, 208849, 212521, 214369, 218089, 229441, 237169, 241081, 249001, 253009, 259081, 262144, 271441, 273529, 279841, 292681, 299209, 310249, 316969, 323761, 326041, 332929, 344569, 351649, 358801, 361201, 368449, 375769, 380689, 38316

In [196]:

for x in range(-2,2):
    print("x:"+ str(x))
    print(f(x))

x:-2
5
x:-1
2
x:0
1
x:1
2
